In [1]:
import pandas as pd
import glob
import codecs
import re

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util

client_id = '55e915e6dc2549b29feb68ef4e611e2a'
client_secret = '086669968e7e4892bc08c8d39308afaf'
username="21ulbu67tca2mhvh2jiy3g4da"

In [2]:
token = util.prompt_for_user_token(
    username="21ulbu67tca2mhvh2jiy3g4da",
    scope='user-read-playback-state,playlist-read-private,user-modify-playback-state,playlist-modify-public', 
    client_id = '55e915e6dc2549b29feb68ef4e611e2a',
    client_secret = '086669968e7e4892bc08c8d39308afaf',
    redirect_uri="http://localhost:8889/callback"
)
sp = spotipy.Spotify(auth=token)

In [3]:
def fix_Play_List(Play_List):
    Play_List["アーティスト"] = Play_List["アーティスト"].str.replace('Remix', '')
    Play_List["アーティスト"] = Play_List["アーティスト"].str.replace('ft.', '/')
    Play_List["アーティスト"] = Play_List["アーティスト"].str.replace('Feat.', '/')
    Play_List["アーティスト"] = Play_List["アーティスト"].str.replace('feat.', '/')
    Play_List["アーティスト"] = Play_List["アーティスト"].str.replace('vs', '/')
    Play_List["アーティスト"] = Play_List["アーティスト"].str.replace('remix', '/')
    Play_List["アーティスト"] = Play_List["アーティスト"].str.replace('&', '/')
    Play_List["アーティスト"] = Play_List["アーティスト"].str.replace('(', '/')
    Play_List["アーティスト"] = Play_List["アーティスト"].str.replace(')', '/')
    Play_List["アーティスト"] = Play_List["アーティスト"].str.replace('[', '')
    Play_List["アーティスト"] = Play_List["アーティスト"].str.replace(']', '')


    #Play_List["トラックタイトル"] = Play_List["トラックタイトル"].str.replace('(Original Mix)', '')
    Play_List["トラックタイトル"] = Play_List["トラックタイトル"].str.replace('(Music Video HD)', '')
    Play_List["トラックタイトル"] = Play_List["トラックタイトル"].str.replace("Original Mix", '')
    Play_List["トラックタイトル"] = Play_List["トラックタイトル"].str.replace("Extended Mix", '')
    Play_List["トラックタイトル"] = Play_List["トラックタイトル"].str.replace("-320", '')
    Play_List["トラックタイトル"] = Play_List["トラックタイトル"].str.replace("official audio", '')
    Play_List["トラックタイトル"] = Play_List["トラックタイトル"].str.replace("(", '')
    Play_List["トラックタイトル"] = Play_List["トラックタイトル"].str.replace(")", '')
    Play_List["トラックタイトル"] = Play_List["トラックタイトル"].str.replace("[", '')
    Play_List["トラックタイトル"] = Play_List["トラックタイトル"].str.replace("]", '')
    
    return Play_List

In [5]:
def get_id_track_artist(name):
    id_list = []
    name_list = []
    artists = []
    d = ""
    result = sp.search(q='track:'+name, limit=50, offset=0, type='track', market=None)


    for track in result['tracks']['items']:
        id_list.append(track['id'])
        name_list.append(track["name"])
        #artists.append(track["artists"][0]['name'])
        if len(track["artists"])== 1:
            d = track["artists"][0]['name']
        if len(track["artists"])== 2:
            d = track["artists"][0]['name'] + "/" + track["artists"][1]['name']
        if len(track["artists"]) >2:
            d = track["artists"][0]['name'] + "/" + track["artists"][1]['name']+ "/" + track["artists"][2]['name']
        artists.append(d)
    
    df = pd.DataFrame({'id': id_list ,'track': name_list, "artist" : artists})
    
    return df


In [6]:
def get_appropriate_track(df,P):
    match_list = []
    for j in range(len(df)):
        wi = 0
        for w in re.split('[/]',df["artist"][j]) :
            for p_w in re.split('[/]',P):
                #print(w)
                #print(p_w)
                if w in p_w:
                    #print("fit")
                    wi = wi +1
        match_list.append(wi)

    df["match"] = match_list

    df = df.sort_values(by = "match", ascending=False)
    df.head()
    return df.iloc[0:1,:]        

In [7]:
def get_track_info(play_list_df):
    
    columns=['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
           'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
           'type', 'id', 'uri', 'track_href', 'analysis_url', 'duration_ms',
           'time_signature']
    features_df = pd.DataFrame(index=[], columns=columns)

    for idx in play_list_df["id"]:
        features = sp.audio_features(idx)
        features_temp = pd.DataFrame(features)
        features_df = pd.concat([features_df, features_temp])
    features_df = features_df.reset_index()

    return features_df


In [8]:
files = glob.glob("./playlists/*.txt")

In [9]:
for file in files:
    with  codecs.open(file, 'r', 'utf-16', 'ignore') as f:
        Play_List = pd.read_table(f) 
        Play_List = fix_Play_List(Play_List)
        
    cols = ['id', 'track',"artist", "match"]
    play_list_df = pd.DataFrame(index=[], columns=cols)
    for Play_List_num in range(len(Play_List)):
        name = Play_List["トラックタイトル"][Play_List_num]
        df = get_id_track_artist(name)
        appropriate_track = get_appropriate_track(df, Play_List["アーティスト"][Play_List_num])
        play_list_df = pd.concat([play_list_df, appropriate_track])
    play_list_df = play_list_df.reset_index()

    features_df = get_track_info(play_list_df)

    play_list_df = pd.merge(play_list_df, features_df, on='id')

    play_list_df.to_csv(file[:-4] + ".csv")